<h1> Capstone Project Week 1

<h2> Review Criteria

<h3>Part 1: A description of the problem and a discussion of the background. (15 marks)

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

<h3> Part 2: A description of the data and how it will be used to solve the problem. (15 marks)

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

<h2> Part 1 (Problem)

Where in Downtown Sacramento, California would it be best to open a restaurant? 

Questions to answer:
What are the most common types of restaurants?\
What are the least common types of restaurants?\
Where are the most popular restaurants? i.e. What restaurants have the most reviews?


<h2> Part 2 (Data)

The data that will be used is Foursquare data for Sacramento, California. It will focus on venues of restaurant types. The Popularity will be a good indicator for how good an area is for a restaurant 

In [1]:
#importing libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests, json

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import requests

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
# Get longitude and latitude for Sacramento
address = 'Capitol Park Sacramento, California'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sacramento is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sacramento is 38.5760675, -121.49144704301602.


In [3]:
CLIENT_ID = 'ZTBWDEYHN1G2TQTPDGVP0CZXBVUXZUVA0KYT3SRZ4TTLIXVB' # your Foursquare ID
CLIENT_SECRET = '5IBVWVLZJH2UY2JSFVBM4VXYO1RSEN22EZ00AOSEL02HNSWR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 3000 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
#categoryId = '4bf58dd8d48988d148941735'
search_query = 'italian'
# create URL with search query
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

#creat radius w/o search query
#url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius, LIMIT, catagoryId)

In [4]:
results = requests.get(url).json()


In [5]:
#venues = results['response']['groups'][0]['items'] #explore
venues = results['response']['venues'] #search
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4f326e7e19836c91c7d72f4e,Italian Imports,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1593984788,False,1100 Front St,38.582260,-121.506477,"[{'label': 'display', 'lat': 38.58226013183594...",1478,95814,US,Sacramento,CA,United States,"[1100 Front St, Sacramento, CA 95814, United S...",NaN
1,4dd84af5fa76ad96d13af3d3,Little Rich's Italian Ice,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",v-1593984788,False,106 L St,38.581032,-121.506081,"[{'label': 'display', 'lat': 38.58103179931640...",1388,95814,US,Sacramento,CA,United States,"[106 L St (at Front St), Sacramento, CA 95814,...",at Front St
2,5e0230225a57b600084eaead,Sacramento Cucina Italiana,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1593984788,False,400 Capitol Mall,38.578685,-121.502567,"[{'label': 'display', 'lat': 38.5786852130786,...",1010,95814,US,Sacramento,CA,United States,"[400 Capitol Mall, Sacramento, CA 95814, Unite...",NaN


In [10]:
#for explore put "venue._____" i.e venue.names
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
dataframe_filtered = nearby_venues

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,id,name,categories,referralId,hasPerk,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet
0,4f326e7e19836c91c7d72f4e,Italian Imports,Food,v-1593984788,False,1100 Front St,38.582260,-121.506477,"[{'label': 'display', 'lat': 38.58226013183594...",1478,95814,US,Sacramento,CA,United States,"[1100 Front St, Sacramento, CA 95814, United S...",NaN
1,4dd84af5fa76ad96d13af3d3,Little Rich's Italian Ice,Ice Cream Shop,v-1593984788,False,106 L St,38.581032,-121.506081,"[{'label': 'display', 'lat': 38.58103179931640...",1388,95814,US,Sacramento,CA,United States,"[106 L St (at Front St), Sacramento, CA 95814,...",at Front St
2,5e0230225a57b600084eaead,Sacramento Cucina Italiana,Italian Restaurant,v-1593984788,False,400 Capitol Mall,38.578685,-121.502567,"[{'label': 'display', 'lat': 38.5786852130786,...",1010,95814,US,Sacramento,CA,United States,"[400 Capitol Mall, Sacramento, CA 95814, Unite...",NaN


In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Capitol Park Center

# add a red circle marker to represent city center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Capitol Park Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

